## 1️⃣ Setup: Configure Dataset & CV Strategy

In [ ]:
import os
import sys
from pathlib import Path
import yaml

# ========== CONFIGURATION ==========
DATASET_NAME = "my-hull-models"  # ← 업로드한 dataset 이름

# ========== CV STRATEGY 선택 (여기만 바꾸세요!) ==========
CV_STRATEGY = "expanding_window"  # time_based, expanding_window, purged_walk_forward, regime_aware
# ====================================================================

print("="*80)
print(f"CV STRATEGY EVALUATION: {CV_STRATEGY.upper()}")
print("="*80)

# Kaggle 데이터셋 경로
DATASET_PATH = Path(f"/kaggle/input/{DATASET_NAME}")

if DATASET_PATH.exists():
    sys.path.insert(0, str(DATASET_PATH))
    print(f"✓ Dataset found: {DATASET_PATH}")
else:
    print(f"❌ Dataset not found: {DATASET_PATH}")
    input_dir = Path("/kaggle/input/")
    if input_dir.exists():
        print("\n📁 Available datasets:")
        for item in input_dir.iterdir():
            print(f"  - {item.name}")
    raise FileNotFoundError(f"Dataset '{DATASET_NAME}' not found!")

# Config 복사
config_path = DATASET_PATH / "conf" / "params.yaml"
if config_path.exists():
    working_config_dir = Path("/kaggle/working/conf")
    working_config_dir.mkdir(parents=True, exist_ok=True)
    
    import shutil
    shutil.copy(config_path, working_config_dir / "params.yaml")
    sys.path.insert(0, str(working_config_dir.parent))
    print(f"✓ Config copied to: {working_config_dir}/params.yaml")
else:
    print(f"⚠️  Config file not found: {config_path}")

print(f"\n✅ Setup complete! Using CV strategy: {CV_STRATEGY}")

CV STRATEGY EVALUATION: TIME_BASED
❌ Dataset not found: /kaggle/input/my-hull-models


FileNotFoundError: Dataset 'my-hull-models' not found!

## 2️⃣ Train Models with Selected CV Strategy

In [2]:
import warnings

import numpy as np
import pandas as pd
import polars as pl
import lightgbm as lgb
import pickle
import json
from typing import List, Tuple

from src.data import DataLoader
from src.features import FeatureEngineering
from src.cv import create_cv_strategy
from src.metric import CompetitionMetric
from src.position import SharpeScalingMapper
from src.tuner import OptunaLightGBMTuner

warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

print("="*80)
print(f"TRAINING WITH {CV_STRATEGY.upper()} CV")
print("="*80)

# ========== 1. Data Preparation ==========
print("\n📊 Step 1: Loading data...")
data_loader = DataLoader("conf/params.yaml")
train_df, _ = data_loader.load_data()
print(f"✓ Loaded {len(train_df)} samples")

print("\n🔧 Step 2: Preprocessing...")
train_processed, _ = data_loader.preprocess_timeseries(
    train_df,
    handle_outliers=False,
    normalize=False,
    scale=False,
    window=60
)
is_feature_engineering = True  # Feature Engineering 수행 여부
is_hyperparameter_tuning = False  # Hyperparameter Tuning 수행 여부 (빠른 테스트용)
n_trials = 1

# ========== 1.5 Prepare realized volatility target ==========
# Sort by date_id to ensure chronological order
train_processed = train_processed.sort_values('date_id').reset_index(drop=True)

# Calculate rolling volatility over time
train_processed['realized_vol'] = (
    train_processed['forward_returns']
    .rolling(window=30, min_periods=5)
    .std()
)

# Fill remaining NaN values with mean
train_processed['realized_vol'] = train_processed['realized_vol'].fillna(
    train_processed['realized_vol'].mean()
)

print(f"✓ Realized volatility calculated (rolling window=30)")
print(f"  Mean: {train_processed['realized_vol'].mean():.6f}")
print(f"  Std: {train_processed['realized_vol'].std():.6f}")
print(f"  NaN count: {train_processed['realized_vol'].isna().sum()}")

# ========== 1.6 Feature Engineering & Selection ==========
print("\n🔧 Step 2.5: Feature engineering and selection...")

# Define metadata columns upfront
metadata_cols = ['date_id', 'forward_returns', 'realized_vol']
if 'risk_free_rate' in train_processed.columns:
    metadata_cols.append('risk_free_rate')

if not is_feature_engineering:
    print("⚠️  피처 엔지니어링을 건너뜁니다 (전처리된 feature만 사용)")
    
    # All other columns are features
    all_feature_cols = [col for col in train_processed.columns if col not in metadata_cols]
    
    # Return model: uses all features
    return_feature_cols = all_feature_cols
    
    # Risk model: exclude forward_returns to prevent data leakage
    risk_feature_cols = [col for col in all_feature_cols if col != 'forward_returns']
    
    # Use original data
    train_df_final = train_processed
    
    print(f"✓ Return 모델: {len(return_feature_cols)}개 feature")
    print(f"✓ Risk 모델: {len(risk_feature_cols)}개 feature (forward_returns 제외)")
    
else:
    print("✅ 피처 엔지니어링 & 선택 수행")
    
    # Feature engineering
    feature_engineer = FeatureEngineering("conf/params.yaml")
    df_engineered = feature_engineer.fit_transform(train_processed)
    
    # Feature selection for return model
    df_selected, _ = feature_engineer.select_features_by_importance(
        df_engineered, 
        target_col='forward_returns'
    )
    df_final, _ = feature_engineer.remove_correlated_features(
        df_selected, 
        target_col='forward_returns'
    )
    
    # Get selected feature columns (everything except metadata)
    selected_features = [col for col in df_final.columns if col not in metadata_cols]
    
    print(f"\n🔍 디버깅: Feature selection 결과")
    print(f"  df_final shape: {df_final.shape}")
    print(f"  selected_features: {len(selected_features)}개")
    
    # Build final dataframe: selected features from df_final + metadata from train_processed
    train_df_final = df_final[selected_features].copy()
    
    # Add metadata/targets from original train_processed
    for col in metadata_cols:
        if col in train_processed.columns:
            train_df_final[col] = train_processed[col].values
    
    # Return model: uses all selected features
    return_feature_cols = selected_features
    
    # Risk model: exclude forward_returns to prevent data leakage
    risk_feature_cols = [col for col in selected_features if col != 'forward_returns']
    
    print(f"\n✅ 최종 데이터프레임 구성")
    print(f"  Shape: {train_df_final.shape}")
    print(f"  Return features: {len(return_feature_cols)}개")
    print(f"  Risk features: {len(risk_feature_cols)}개")
    print(f"  Metadata columns: {[c for c in metadata_cols if c in train_df_final.columns]}")


# ========== 2. Create CV Strategy ==========
print(f"\n🔀 Step 3: Creating {CV_STRATEGY} CV strategy...")
cv_strategy = create_cv_strategy(
    config_path="conf/params.yaml",
    strategy=CV_STRATEGY
)
folds = list(cv_strategy.get_folds(train_df_final))
print(f"✓ Created {len(folds)} folds")


# ========== 3. Hyperparameter Tuning ==========
if is_hyperparameter_tuning:
    print("\n🔍 Step 4: Hyperparameter tuning...")

    print("\n  📈 Tuning return model...")
    return_tuner = OptunaLightGBMTuner(config_path="conf/params.yaml", n_trials=n_trials, objective_type='return_ic')
    best_params = return_tuner.optimize(
        df=train_df_final,
        feature_cols=return_feature_cols,
        target_col='forward_returns',
        study_name=f"{CV_STRATEGY}_return_tuning",
    )

    return_params = best_params if best_params else {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.8,
        'verbosity': -1,
        'n_jobs': 1
    }

    print("\n  📉 Tuning risk model...")
    risk_tuner = OptunaLightGBMTuner(config_path="conf/params.yaml", n_trials=n_trials, objective_type='risk_correlation')
    best_risk_params = risk_tuner.optimize(
        df=train_df_final,
        feature_cols=risk_feature_cols,
        target_col='realized_vol',
        study_name=f"{CV_STRATEGY}_risk_tuning",
    )

    risk_params = best_risk_params if best_risk_params else {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 15,
        'learning_rate': 0.05,
        'feature_fraction': 0.8,
        'verbosity': -1,
        'n_jobs': 1
    }
    
    print(f"\n✓ Hyperparameter tuning complete!")
else:
    print("\n⚠️  Step 4: Skipping hyperparameter tuning, using default parameters...")
    
    # Default parameters for return model
    return_params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.8,
        'verbosity': -1,
        'n_jobs': 1
    }
    
    # Default parameters for risk model
    risk_params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 15,
        'learning_rate': 0.05,
        'feature_fraction': 0.8,
        'verbosity': -1,
        'n_jobs': 1
    }
    
    print(f"✓ Using default parameters")

# ========== 4. Train Models ==========
print("\n🎯 Step 5: Training models with selected parameters...")
print(f"\n🔍 학습 전 최종 체크:")
print(f"  train_df_final shape: {train_df_final.shape}")
print(f"  train_df_final dtypes: {train_df_final.dtypes.value_counts().to_dict()}")
print(f"  return_feature_cols: {len(return_feature_cols)}개")
print(f"  risk_feature_cols: {len(risk_feature_cols)}개")
print(f"  forward_returns in df: {'forward_returns' in train_df_final.columns}")
print(f"  realized_vol in df: {'realized_vol' in train_df_final.columns}")

return_models = []
risk_models = []
oof_predictions = {
    'r_hat': np.zeros(len(train_df_final)),
    'sigma_hat': np.zeros(len(train_df_final)),
    'allocations': np.zeros(len(train_df_final)),
    'mask': np.zeros(len(train_df_final), dtype=bool)
}

# Store per-fold scores
fold_scores = []

for fold_idx, (train_idx, val_idx) in enumerate(folds):
    print(f"\n  📁 Fold {fold_idx + 1}/{len(folds)}")
    
    # Return model
    X_train = train_df_final.iloc[train_idx][return_feature_cols]
    y_train = train_df_final.iloc[train_idx]['forward_returns']
    X_val = train_df_final.iloc[val_idx][return_feature_cols]
    y_val = train_df_final.iloc[val_idx]['forward_returns']
    
    print(f"    Return model - X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"    X_train dtypes: {X_train.dtypes.value_counts().to_dict()}")
    print(f"    X_train sample stats:")
    print(f"      - NaN counts: {X_train.isna().sum().sum()}")
    print(f"      - Inf counts: {np.isinf(X_train.values).sum()}")
    print(f"      - Constant cols (std=0): {(X_train.std() == 0).sum()}")
    print(f"      - First 5 features: {X_train.columns[:5].tolist()}")
    print(f"      - Sample values (first row, first 5 cols): {X_train.iloc[0, :5].values}")
    
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
    
    return_model = lgb.train(
        return_params,
        train_data,
        num_boost_round=100,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(stopping_rounds=20, verbose=False)]
    )
    return_models.append(return_model)
    r_hat = return_model.predict(X_val)
    
    # Risk model
    X_train_risk = train_df_final.iloc[train_idx][risk_feature_cols]
    y_train_vol = train_df_final.iloc[train_idx]['realized_vol']
    X_val_risk = train_df_final.iloc[val_idx][risk_feature_cols]
    y_val_vol = train_df_final.iloc[val_idx]['realized_vol']
    
    print(f"    Risk model - X_train shape: {X_train_risk.shape}, y_train shape: {y_train_vol.shape}")
    
    train_data_vol = lgb.Dataset(X_train_risk, label=y_train_vol)
    val_data_vol = lgb.Dataset(X_val_risk, label=y_val_vol, reference=train_data_vol)
    
    risk_model = lgb.train(
        risk_params,
        train_data_vol,
        num_boost_round=100,
        valid_sets=[val_data_vol],
        callbacks=[lgb.early_stopping(stopping_rounds=20, verbose=False)]
    )
    risk_models.append(risk_model)
    sigma_hat = np.maximum(risk_model.predict(X_val_risk), 1e-6)
    
    # Position strategy
    position_mapper = SharpeScalingMapper("conf/params.yaml")
    allocations = position_mapper.map_positions(
        r_hat=r_hat,
        sigma_hat=sigma_hat,
        k=1.0,
        b=2.0
    )
    
    # Store OOF
    oof_predictions['r_hat'][val_idx] = r_hat
    oof_predictions['sigma_hat'][val_idx] = sigma_hat
    oof_predictions['allocations'][val_idx] = allocations
    oof_predictions['mask'][val_idx] = True
    
    # Calculate fold score
    metric_calc = CompetitionMetric()
    fold_forward_returns = train_df_final.iloc[val_idx]['forward_returns'].values
    fold_risk_free = train_df_final.iloc[val_idx]['risk_free_rate'].values if 'risk_free_rate' in train_df_final.columns else None
    
    fold_score = metric_calc.calculate_score(
        allocations=allocations,
        forward_returns=fold_forward_returns,
        risk_free_rate=fold_risk_free
    )
    fold_scores.append(fold_score)
    
    print(f"    Return RMSE: {np.sqrt(np.mean((y_val - r_hat)**2)):.6f}")
    print(f"    Risk RMSE: {np.sqrt(np.mean((y_val_vol - sigma_hat)**2)):.6f}")
    print(f"    Fold Score: {fold_score['score']:.6f} (Sharpe: {fold_score['sharpe']:.6f})")

# ========== 5. Calculate OOF Score ==========
print("\n📊 Step 6: Calculating OOF competition score...")

metric_calc = CompetitionMetric()
oof_mask = oof_predictions['mask']

oof_score = metric_calc.calculate_score(
    allocations=oof_predictions['allocations'][oof_mask],
    forward_returns=train_df_final['forward_returns'].values[oof_mask],
    risk_free_rate=train_df_final['risk_free_rate'].values[oof_mask] if 'risk_free_rate' in train_df_final.columns else None
)

print("\n" + "="*80)
print(f"OOF SCORE - {CV_STRATEGY.upper()}")
print("="*80)
print(f"\n📊 Overall OOF Score:")
print(f"Competition Score: {oof_score['score']:.6f}")
print(f"  → Sharpe Ratio: {oof_score['sharpe']:.6f}")
print(f"  → Vol Penalty: {oof_score['vol_penalty']:.4f}")
print(f"  → Return Penalty: {oof_score['return_penalty']:.4f}")
print(f"  → Vol Ratio: {oof_score['vol_ratio']:.4f}")
print(f"Coverage: {oof_mask.sum() / len(train_df_final):.2%}")

print(f"\n📈 Per-Fold Scores:")
print("-" * 80)
fold_score_values = [fs['score'] for fs in fold_scores]
fold_sharpe_values = [fs['sharpe'] for fs in fold_scores]
for i, fs in enumerate(fold_scores):
    print(f"Fold {i+1}: Score={fs['score']:.6f}, Sharpe={fs['sharpe']:.6f}, Vol Penalty={fs['vol_penalty']:.4f}")
print("-" * 80)
print(f"Mean ± Std: {np.mean(fold_score_values):.6f} ± {np.std(fold_score_values):.6f}")
print(f"Min / Max: {np.min(fold_score_values):.6f} / {np.max(fold_score_values):.6f}")
print(f"Sharpe Mean ± Std: {np.mean(fold_sharpe_values):.6f} ± {np.std(fold_sharpe_values):.6f}")
print("="*80)

# ========== 6. Save Models & OOF Score ==========
print("\n💾 Step 7: Saving models and results...")

model_dir = Path("/kaggle/working/artifacts")
model_dir.mkdir(parents=True, exist_ok=True)

# Save return models
return_dir = model_dir / "return_models"
return_dir.mkdir(exist_ok=True)
for i, model in enumerate(return_models):
    with open(return_dir / f"model_fold_{i}.pkl", 'wb') as f:
        pickle.dump(model, f)

# Save risk models
risk_dir = model_dir / "risk_models"
risk_dir.mkdir(exist_ok=True)
for i, model in enumerate(risk_models):
    with open(risk_dir / f"model_fold_{i}.pkl", 'wb') as f:
        pickle.dump(model, f)

# Save feature names for both return and risk models
feature_info = {
    'return_features': return_models[0].feature_name(),
    'risk_features': risk_models[0].feature_name()
}
with open(model_dir / "feature_names.json", 'w') as f:
    json.dump(feature_info, f)

# Save OOF score with fold-level details
with open(model_dir / "oof_score.json", 'w') as f:
    json.dump({
        'cv_strategy': CV_STRATEGY,
        'oof_score': float(oof_score['score']),
        'oof_sharpe': float(oof_score['sharpe']),
        'oof_vol_penalty': float(oof_score['vol_penalty']),
        'oof_return_penalty': float(oof_score['return_penalty']),
        'oof_vol_ratio': float(oof_score['vol_ratio']),
        'coverage': float(oof_mask.sum() / len(train_df_final)),
        'fold_scores': [
            {
                'fold': i + 1,
                'score': float(fs['score']),
                'sharpe': float(fs['sharpe']),
                'vol_penalty': float(fs['vol_penalty']),
                'return_penalty': float(fs['return_penalty']),
                'vol_ratio': float(fs['vol_ratio'])
            }
            for i, fs in enumerate(fold_scores)
        ],
        'fold_score_stats': {
            'mean': float(np.mean(fold_score_values)),
            'std': float(np.std(fold_score_values)),
            'min': float(np.min(fold_score_values)),
            'max': float(np.max(fold_score_values)),
            'sharpe_mean': float(np.mean(fold_sharpe_values)),
            'sharpe_std': float(np.std(fold_sharpe_values))
        }
    }, f, indent=2)

print(f"✓ Models saved to {model_dir}")
print(f"✓ Feature names saved:")
print(f"  - Return features: {len(feature_info['return_features'])}")
print(f"  - Risk features: {len(feature_info['risk_features'])}")
print(f"✓ OOF score saved: {oof_score['score']:.6f}")
print(f"✓ Fold-level scores saved")
print("\n✅ Training complete!")


ModuleNotFoundError: No module named 'polars'

## 3️⃣ Load Models & Start Inference Server

In [ ]:
# ========== Cell 3: Inference - Competition Server or Local Parquet ==========

# Load feature names first
with open(model_dir / "feature_names.json", 'r') as f:
    feature_info = json.load(f)
    return_feature_names = feature_info['return_features']
    risk_feature_names = feature_info['risk_features']

def predict(test, model=None):
    """
    Prediction function for competition server.
    
    Args:
        test: polars DataFrame with test features
        model: Not used (models loaded globally)
    
    Returns:
        float: allocation (0 to 2)
    """
    try:
        # Convert to pandas
        test_pd = test.to_pandas()
        
        # Preprocess test data (동일한 전처리 적용)
        data_loader = DataLoader("conf/params.yaml")
        test_processed, _ = data_loader.preprocess_timeseries(
            test_pd,
            handle_outliers=True,
            normalize=True,
            scale=True,
            window=60
        )
        
        # Prepare features for RETURN model (in exact training order)
        X_test_return = pd.DataFrame(index=test_processed.index)
        for feat in return_feature_names:
            if feat in test_processed.columns:
                X_test_return[feat] = test_processed[feat]
            else:
                X_test_return[feat] = 0.0
        
        # Prepare features for RISK model (in exact training order)
        X_test_risk = pd.DataFrame(index=test_processed.index)
        for feat in risk_feature_names:
            if feat in test_processed.columns:
                X_test_risk[feat] = test_processed[feat]
            else:
                X_test_risk[feat] = 0.0
        
        # Ensemble prediction - return (FIX: 올바른 앙상블)
        r_hat_preds = np.array([model.predict(X_test_return) for model in return_models])
        r_hat = float(np.mean(r_hat_preds, axis=0)[0])
        
        # Ensemble prediction - risk (FIX: 올바른 앙상블)
        sigma_hat_preds = np.array([model.predict(X_test_risk) for model in risk_models])
        sigma_hat = float(np.mean(sigma_hat_preds, axis=0)[0])
        sigma_hat = max(sigma_hat, 1e-6)
        
        # Position mapping
        mapper = SharpeScalingMapper()
        allocation = mapper.map_positions(
            r_hat=np.array([r_hat]),
            sigma_hat=np.array([sigma_hat]),
            k=1.0,
            b=2.0
        )[0]
        
        # Ensure bounds
        allocation = max(0.0, min(2.0, float(allocation)))
        
        return allocation
        
    except Exception as e:
        print(f"⚠️  Prediction error: {e}")
        import traceback
        traceback.print_exc()
        return 0.0

print("✓ Prediction function defined!")

# ========== Check if running in Kaggle environment ==========
import os

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    # ========== COMPETITION MODE: Start Inference Server ==========
    print("\n" + "="*80)
    print("🚀 COMPETITION MODE - STARTING INFERENCE SERVER")
    print("="*80)
    print(f"CV Strategy: {CV_STRATEGY.upper()}")
    print(f"OOF Score: {oof_score['score']:.6f}")
    print(f"Models: {len(return_models)} folds")
    print("="*80)
    
    import kaggle_evaluation.default_inference_server
    
    inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)
    inference_server.serve()
    
    print("\n✅ Inference server completed!")

else:
    # ========== LOCAL MODE: Generate submission.parquet ==========
    print("\n" + "="*80)
    print("💻 LOCAL MODE - GENERATING SUBMISSION.PARQUET")
    print("="*80)
    
    # Auto-detect test data path
    input_dir = Path("/kaggle/input/")
    test_path = None
    
    # Search for test.csv in all input datasets
    if input_dir.exists():
        for dataset_dir in input_dir.iterdir():
            if dataset_dir.is_dir():
                potential_test = dataset_dir / "test.csv"
                if potential_test.exists():
                    test_path = potential_test
                    print(f"✓ Found test data: {test_path}")
                    break
    
    if test_path is None or not test_path.exists():
        print("⚠️  Warning: Test file not found, using placeholder")
        submission = pl.DataFrame({
            'date_id': range(100),
            'allocation': [1.0] * 100
        })
    else:
        test_data = pl.read_csv(test_path)
        print(f"✓ Loaded test data: {len(test_data)} rows")
        
        # Generate predictions
        allocations = []
        for idx in range(len(test_data)):
            test_row = test_data[idx:idx+1]
            allocation = predict(test_row)
            allocations.append(allocation)
        
        # Create submission
        submission = pl.DataFrame({
            'date_id': test_data['date_id'].to_list(),
            'allocation': allocations
        })
    
    # Save to parquet
    output_path = Path("/kaggle/working/submission.parquet")
    submission.write_parquet(output_path)
    
    print(f"\n✅ Submission saved to: {output_path}")
    print(f"📊 Prediction Summary:")
    print(f"  Mean allocation: {submission['allocation'].mean():.6f}")
    print(f"  Std allocation: {submission['allocation'].std():.6f}")
    print(f"  Min allocation: {submission['allocation'].min():.6f}")
    print(f"  Max allocation: {submission['allocation'].max():.6f}")
    print(f"\n🔍 First 5 predictions:")
    print(submission.head())
    print("="*80)